# Lab_4
1. Провести моделирование регрессионного процесса с гетероскедастичным возмущением.
2. Полученные данные проверить по тестам на наличие гетероскедастичности.
3. Провести оценивание параметров регрессионной модели
по доступному обобщенному МНК и по обыкновенному МНК.
4. Сравнить эффективность оценок в этих двух случаях по
квадрату их расстояния до известных истинных значений параметров. 

## Инициализация программы

In [1]:
import numpy as np
import random as rnd
import math

f = lambda x1, x2 : np.array([1, 1/x1, x1, x2, x2**2]) 
teta = np.array([0.001, 1, 0.001, 0.0001, 1])
u = lambda x1, x2: 0.001 + 1/x1 + x1* 0.001 + x2*0.0001 + x2**2

# Генерирует точки (120)
def generateData():
    import random as rnd

    x1_points = []
    x2_points = []

    for s in range(0, 120):
        x2_points.append((rnd.random() - 0.5) * 2)

    for s in range(0, 60):
        x1_points.append(-rnd.random() * 5 - 1)
        x1_points.append(rnd.random() * 5 + 1)
    return x1_points, x2_points

# Строит модель из 2 лабы
def lab_2(f, teta, u, x1_array, x2_array, p):   
    vector_u = list()

    # Рассчиатли U
    for i in range(len(x1_array)):
        vector_u.append(u(x1_array[i], x2_array[i]))

    vect_arr_u = np.array(vector_u)
    average_u = vect_arr_u.mean()

    w2 = np.dot(vect_arr_u - average_u, vect_arr_u - average_u) / (len(vect_arr_u - 1))  
    sig = p * (w2)**0.5

    # Сгенерировали U с помехами (y)
    y = []
    for i in range(len(x1_array)):
        y.append(vector_u[i] + np.random.normal(0, sig))
    
    # Построение матрицы X
    X = []
    for i in range(len(teta)):
        X.append([])
        for j in range(len(x1_array)):
            X[i].append(f(x1_array[j], x2_array[j])[i])

    X = np.matrix(X)

    Q_ = X.dot(X.T)
    Q_ = np.linalg.inv(Q_)
    Q_ = Q_.dot(X)
    Q_ = Q_ * np.matrix(y).T
    
    e = []
    e = y - (X.T).dot(np.matrix(teta).T).T
    sigma2 = float(e.dot(e.T)/(len(vect_arr_u)-len(teta))) # Несмещенная оценка оценка дисперсии

    F = sigma2 / sig**2
        
    return Q_, F, X, sigma2, y, vector_u

## 1. Провести моделирование регрессионного процесса с гетероскедастичным возмущением.

In [2]:
x1_arr, x2_arr = generateData()
Q_, F, X, sigma2, y, u_arr = lab_2(f, teta, u, x1_arr, x2_arr, 0.07)

def someVal(x):
    l = [];
    for i in range(len(x)):
        l.append(float(math.exp(abs(x[i]))))
    l = np.array(l, dtype = float)
    return l

z = np.array([1, someVal(y)])

## 2. Полученные данные проверить по тестам на наличие гетероскедастичности.
### 1) Тест Бреуша-Пагана

In [3]:
e_t = np.array(y) - np.array(u_arr)
sigma_tilda_2 = sum(e_t**2) / 120

someConst = e_t**2/sigma_tilda_2

mean_val = someConst.mean()
ESS = 0

for i in range(len(someConst)):
    ESS+=(someConst[i] - mean_val)**2

from scipy.stats import chi2
value = chi2.ppf(0.95, len(someVal(y)))

if ESS/2 < value:
    print('Гипотеза о гомоскедастичности принимается.')
else:
    print('Гипотеза о гомоскедастичности отвергается.')

Гипотеза о гомоскедастичности принимается.


### 2) Тест Голдфельда-Квандтона

In [4]:
import pandas as pd

def someFunc(x1, x2):
    l = []
    for i in range(len(x1)):
        l.append(x1[i]**2 + x2[i]**2)
    l = np.array(l)
    return l
    
values = pd.DataFrame({'x1': x1_arr, 'x2': x2_arr, 'y': y, 'u': u_arr, 'x**2': someFunc(x1_arr, x2_arr)})

# Сортировка по возрастанию квадрата
values = values.sort_values(by=['x**2'])
values = values.reset_index(drop=True)

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
nc = len(x1_arr)/3
# Удалить nc центральную выборку
newTable = values.drop(np.r_[nc:119 - nc])

# Оценка по первым (n-nc)/2 наблюдений
X = []
for i in range(len(teta)):
    X.append([])
    for j in range(int(nc)):
        X[i].append(f(newTable['x1'][j],newTable['x2'][j])[i])
X = np.matrix(X)

Q_T = np.linalg.inv(X.dot(X.T))
Q_T = Q_T.dot(X)
Q_T = Q_T.dot(np.matrix(newTable['y'][0:int(nc)]).T)

RSS1=(np.array(newTable['y'][0:int(nc)])-X.T.dot(Q_T).T)
RSS1=float(RSS1.dot(RSS1.T))

# Оценка по последним (n-nc)/2 наблюдений
n = len(x1_arr)
X = []

for i in range(len(teta)):
    X.append([])
    for j in range(int(n - nc - 1), int(n - 1), 1):
        X[i].append(f(newTable['x1'][j],newTable['x2'][j])[i])
X = np.matrix(X)

Q_T = np.linalg.inv(X.dot(X.T))
Q_T = Q_T.dot(X)

newTable = newTable.reset_index(drop=True)
Q_T = Q_T.dot(np.matrix(newTable.y[40:80]).T)
RSS2=(np.array(newTable['y'][40:80])-X.T.dot(Q_T).T)
RSS2=float(RSS2.dot(RSS2.T))


from scipy.stats import f
k = 5
value = f.ppf(0.95, (n - nc - 2*k)/2, (n - nc - 2*k) / 2)
if RSS1/RSS2 < value:
    print('Гипотеза о гомоскедастичности принимается.')
else:
    print('Гипотеза о гомоскедастичности отвергается.')

Гипотеза о гомоскедастичности принимается.


## 3. Провести оценивание параметров регрессионной модели по доступному обобщенному МНК и по обыкновенному МНК.

In [14]:
# обобщенный МНК
f = lambda x1, x2 : np.array([1, 1/x1, x1, x2, x2**2]) 

X_new = []
for i in range(len(teta)):
    X_new.append([])
    for j in range(len(x1_arr)):
        X_new[i].append(f(x1_arr[j], x2_arr[j])[i])

X_new = np.matrix(X_new)
V = np.matrix(np.diag(someVal(y)))
Q_1 = X_new.dot(np.linalg.inv(V))
Q_1 = np.linalg.inv(Q_1.dot(X_new.T))
Q_1 = Q_1.dot(X_new.dot( np.linalg.inv(V)))
Q_1 = Q_1.dot(np.matrix(y).T)

In [19]:
print('Истинные значения', teta)
print('МНК', Q_.T)
print('ОМНК', Q_1.T)

Истинные значения [1.e-03 1.e+00 1.e-03 1.e-04 1.e+00]
МНК [[-7.62359016e-04  9.94902563e-01  1.74470421e-03 -1.04486332e-02
   1.00078487e+00]]
ОМНК [[-0.00223446  0.99000916  0.00199765 -0.00936709  1.00382756]]


## 4. Сравнить эффективность оценок в этих двух случаях по квадрату их расстояния до известных истинных значений параметров.

In [31]:
print('Сравнение МНК с истиной', np.linalg.norm(teta - Q_.T)**2)

Сравнение МНК с истиной 0.0001415340422534778


In [32]:
print('Сравнение ОМНК с истиной',np.linalg.norm(teta - Q_1.T)**2)

Сравнение ОМНК с истиной 0.00021554981153097327
